# refellips demo: Solid/Aid
This demo presents an example use case for fitting multi-wavelength multi-angle ellipsometry data with refellips.

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from refnx.analysis import CurveFitter
from refnx.reflect import Slab

In [ ]:
from refellips.dataSE import DataSE, open_EP4file
from refellips.reflect_modelSE import ReflectModelSE
from refellips.objectiveSE import ObjectiveSE
from refellips.structureSE import RI

### Import data

In [ ]:
dname = 'WVASE_example_2nmSiO2_20nmPNIPAM_MultiWavelength.txt'
data = DataSE(data=dname)
np.unique(data.wav)

### Build model

In [ ]:
si     = RI('../materials/silicon.csv')
sio2   = RI('../materials/silica.csv')
PNIPAM = RI('../materials/pnipam.csv')
air    = RI('../materials/air.csv')

PNIPAM_layer = PNIPAM(150)
PNIPAM_layer.thick.setp(vary=True, bounds=(100,500))

struc = air() | PNIPAM_layer | sio2(20) | si()

model = ReflectModelSE(struc, wavelength=658)

# Fitting experimental data

### Model & data pre fit

In [ ]:
fig, ax = plt.subplots()
axt = ax.twinx()

aois = np.linspace(50,75,100)

for dat in data:
    model.wav = dat._current_wav
    psi, delta = model(aois)
    ax.plot(aois, psi, color='r')
    p = ax.scatter(dat.aoi, dat.psi, color='r')

    axt.plot(aois, delta, color='b')
    d = axt.scatter(dat.aoi, dat.delta, color='b')

    ax.legend(handles=[p,d], labels=['Psi', 'Delta'])
    ax.set(ylabel='Psi', xlabel='AOI, °')
    axt.set(ylabel='Delta')

In [ ]:
objective = ObjectiveSE(model, data)

In [ ]:
objective.residuals()

In [ ]:
fitter = CurveFitter(objective)
fitter.fit(method='least_squares');

### Model & data post fit

In [ ]:
fig, ax = plt.subplots()
axt = ax.twinx()

aois = np.linspace(50,75,100)

for dat in data:
    model.wav = dat._current_wav
    psi, delta = model(aois)
    ax.plot(aois, psi, color='r')
    p = ax.scatter(dat.aoi, dat.psi, color='r')

    axt.plot(aois, delta, color='b')
    d = axt.scatter(dat.aoi, dat.delta, color='b')

    ax.legend(handles=[p,d], labels=['Psi', 'Delta'])
    ax.set(ylabel='Psi', xlabel='AOI, °')
    axt.set(ylabel='Delta')

In [ ]:
for i, x in enumerate(objective.model.parameters):
    print(x)